# Extracting Data From the WMO app

## 1 Insturactions of the WMO

Use the following steps to carry out the project
- Use this link (https://community.wmo.int/en/members/profiles to extract data on
heatwaves warning system/services, see example below, if heatwave is in the list of
warnings provided, then indicate a yes in the provided excel sheet (for the corresponding
country in column B in the excel sheet called “Heatwaves Warning Systems (WMO data)”).


full screenmode : https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9


## 2  Required Libraries

In [116]:
#The webdriver module in Selenium allows for automated control of web browsers.
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


## 3 Opening up the web browser and printing a sample text

The function below opens up chrome browser and waits for 60 minutes to load the data 

input: The website link

Output : the driver and wabdriverwait instance

In [117]:

def openChrome(link):
    # Options customize Chrome WebDriver behavior
    options = Options()

    # Initialize the WebDriver (e.g., Chrome)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Navigate to the desired URL
    driver.get(link)

    # wait for page to load
    wait = WebDriverWait(driver, 60)

    return driver, wait

In [118]:
link = "https://app.powerbi.com/view?r=eyJrIjoiZjNhNzIzM2YtMjRkYS00ZjJjLWEzZmMtNmQzMGQzMDdiODU3IiwidCI6ImVhYTZiZTU0LTQ2ODctNDBjNC05ODI3LWMwNDRiZDhlOGQzYyIsImMiOjl9"
driver_instance, wait  = openChrome(link)

Using the driver instance that we created print an sample text

In [119]:
# Locate all div elements with class "textbox"
div_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="textbox"]')))

# Iterate over the list of elements and print the text content of each
for div_element in div_elements:
    print(div_element.text)
    print("-----")  # Just a separator for clarity

# driver_instance.quit()

Member Profiles
-----
The information contained in these profiles have been provided by WMO Members. The information is based on a self-assessment exercise. The date displayed in each section corresponds with the date in which the information was provided to WMO. 

If you find any issue or inconsistency with the information displayed, please contact community@wmo.int.
-----
Permanent Representative:
-----
Hydrological Adviser:
-----
National Meteorological Service:
-----
Responsible Ministry:
-----


## 4. Clicking on the Early Warning Services 

In [120]:
#EC stands for "Expected Conditions." It is a set of predefined conditions that the Selenium WebDriver uses.
#presence_of_element_located is one of many expected conditions (EC). It checks if an element is present on the DOM of a page.
def wait_and_click(driver, wait, css_selector):
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    driver.execute_script("arguments[0].click();", element)

In [121]:
css_selector = 'a[title=" Early Warning Services"]'
wait_and_click(driver_instance, wait, css_selector)

## 5 - Get Warnings List

In [122]:
def saveWarnings(driver, css_selector):
    #'.title' is the class name of the element that contains the warning text
    warnings= driver_instance.find_elements(By.CSS_SELECTOR, css_selector)
    return warnings

warnings = saveWarnings(driver_instance, '.title')


In [123]:

for warning in warnings:
    print(warning.text)

Dust storm/Sandstorm
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
2 jours
Yes
Yes
No
Yes
Yes


## 6 getting all country names 

In [131]:

def getCountryNames(driver, wait):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    loaded_countries = set()
    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()
    print(loaded_countries)
    # Close the dropdown menu by clicking the dropdown button again
    dropdown.click()
    return loaded_countries

In [132]:
loaded_countries = getCountryNames(driver_instance, wait)

{'', 'Papua New Guinea', 'Singapore', 'Sao Tome and Principe', 'Czechia', 'Djibouti', 'Sri Lanka', 'Burkina Faso', 'Brunei Darussalam', 'British Caribbean Territories - Montserrat', 'Nicaragua', 'Brazil', 'Kuwait', 'Ireland', 'Poland', 'Pakistan', 'Bolivia (Plurinational State of)', 'North Macedonia', 'Tuvalu', 'Morocco', 'Kiribati', 'Seychelles', 'Uzbekistan', 'Egypt', 'Curaçao and Sint Maarten', "Democratic People's Republic of Korea", 'Chad', 'Azerbaijan', 'Chile', 'Zambia', 'British Caribbean Territories - Anguilla', 'Latvia', 'Panama', 'Ukraine', 'Macao, China', 'Saint Lucia', 'Cameroon', 'Croatia', 'Hong Kong, China', 'British Caribbean Territories', 'Malta', 'Kazakhstan', 'United States of America', 'Norway', 'Madagascar', 'Georgia', 'Thailand', 'Cambodia', 'Guyana', 'Bangladesh', 'Iraq', 'Slovakia', 'Costa Rica', 'Ghana', 'United Arab Emirates', "Côte d'Ivoire", 'Tajikistan', 'Mongolia', 'Eritrea', "Lao People's Democratic Republic", 'Micronesia (Federated States of)', 'Malawi'

In [133]:
print(len(loaded_countries ))

191


## 7 Selecting A country

In [135]:
def select_country_from_dropdown(driver, wait, target_country):
    # Open the dropdown menu
    dropdown = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.slicer-dropdown-menu'))
    )
    dropdown.click()

    # Clear the search input
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.searchInput'))
    )
    search_input.clear()

    loaded_countries = set()
    country_selected = False

    while True:
        # Re-fetch the dropdown options in every iteration to avoid StaleElementReferenceException
        current_countries_elements = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.slicerText'))
        )
        
        # Extract country names from the current elements
        current_countries = {element.text for element in current_countries_elements}

        # Check if target country is in the current batch of loaded countries
        if target_country in current_countries:
            for option in current_countries_elements:
                if option.text == target_country:
                    option.click()
                    country_selected = True
                    break

        # If the country has been selected, break out of the loop
        if country_selected:
            break
        
        # Check if no new countries are found
        if not current_countries - loaded_countries:
            break
        
        loaded_countries.update(current_countries)

        # Scroll to the last element to load more countries
        ActionChains(driver).move_to_element(current_countries_elements[-1]).perform()

    return country_selected


In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "United States of America")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "India")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:

warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "Argentina")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
warnings = saveWarnings(driver_instance, '.title')

for warning in warnings:
    print(warning.text)

## selecting 8 country and getting the information

1- Initiate global variables

    - initiate a dictinary

    - driver, wait

2- Load all the country names

3 -Click on Early warning systems

----- repeat this 8 times ----

4- Click on the desired country name

5- extract warning systems in a list

   (if the list is empty)
   
6- save country name and the list in the map

In [95]:
# Convert the set to a list
countries_list = list(loaded_countries)

# # Sort the list
# countries_list.sort()

# print(countries_list)
# test= countries_list[0:8]

In [58]:
size = len(countries_list)//20
slice_list = [countries_list[i:i + size] for i in range(0, len(countries_list), size)]

for slice in slice_list:
    print(slice)

['Papua New Guinea', 'Singapore', 'Sao Tome and Principe', 'Czechia', 'Djibouti', 'Sri Lanka', 'Burkina Faso', 'Brunei Darussalam', 'British Caribbean Territories - Montserrat']
['Nicaragua', 'Brazil', 'Kuwait', 'Ireland', 'Poland', 'Pakistan', 'Bolivia (Plurinational State of)', 'North Macedonia', 'Tuvalu']
['Morocco', 'Kiribati', 'Seychelles', 'Uzbekistan', 'Egypt', 'Curaçao and Sint Maarten', "Democratic People's Republic of Korea", 'Chad', 'Azerbaijan']
['Chile', 'Zambia', 'British Caribbean Territories - Anguilla', 'Latvia', 'Panama', 'Ukraine', 'Macao, China', 'Antigua and Barbuda', 'Saint Lucia']
['Cameroon', 'Croatia', 'Hong Kong, China', 'British Caribbean Territories', 'Malta', 'Kazakhstan', 'United States of America', 'Norway', 'Madagascar']
['Georgia', 'Thailand', 'Cambodia', 'Guyana', 'Afghanistan', 'Bangladesh', 'Iraq', 'Slovakia', 'Costa Rica']
['Ghana', 'Algeria', "Côte d'Ivoire", 'United Arab Emirates', 'Albania', 'Tajikistan', 'Mongolia', 'Eritrea', "Lao People's Demo

In [146]:
import pickle

# # Your list of arrays
# arrays = slice_list

# # Save to a file
# with open('arrays.pkl', 'wb') as f:
#     pickle.dump(arrays, f)



In [147]:
# # Load from a file
with open('arrays.pkl', 'rb') as f:
    loaded_arrays = pickle.load(f)

we can load this list and acces them so that the country list does not change

In [148]:
loaded_arrays[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

lets divide our country list in to 10 arrays



In [59]:
# test_list = ['Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco']
# print(len(test_list))
my_dict = {}

possible fix 

In [60]:
for country in slice_list[0]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)
        


Papua New Guinea
country been selected successfully!
No warnings found for this country.
-----
Singapore
country been selected successfully!
Drought/Dry spell
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Volcanic ash
No
No
No
Yes
Yes
Yes
Yes
Yes
All
Drought/Dry spell
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Tropical cyclone
Volcanic ash
No
No
No
Yes
Yes
Yes
Yes
Yes
All
-----
Sao Tome and Principe
country been selected successfully!
No warnings found for this country.
-----
Czechia
country been selected successfully!
Cold wave
Flood
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Forest fire
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
100000
Cold wave
Flood
Fog
Freezing rain
Frost
Hail
Heat wave
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wild land fire/Fo

In [61]:
my_dict

{'Papua New Guinea': [0],
 'Singapore': ['Drought/Dry spell',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Volcanic ash',
  'No',
  'No',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All'],
 'Sao Tome and Principe': [0],
 'Czechia': ['Cold wave',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100000'],
 'Djibouti': ['Dust storm/Sandstorm',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '6HEURES',
  'Yes',
  'Yes',
  'Yes'],
 'Sri Lanka': [0],
 'Burkina Faso': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/S

array 2 , dictinary 2

In [85]:
my_dict_2 = {}

In [86]:
for country in slice_list[1]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_2[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_2[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)
        


Nicaragua
country been selected successfully!
No warnings found for this country.
-----
Brazil
country been selected successfully!
Cold wave
Drought/Dry spell
Flood
Fog
Frost
Hail
Haze/Smoke
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
All of Brazilian territory is covered by Early Warnings
Cold wave
Drought/Dry spell
Flood
Fog
Frost
Hail
Haze/Smoke
Heat wave
Lightning
Rain/Wet Spell
Thunderstorms/Squall lines
Wind
No
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
All of Brazilian territory is covered by Early Warnings
-----
Kuwait
country been selected successfully!
No warnings found for this country.
-----
Ireland
country been selected successfully!
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Wind
Yes
Yes
Yes
2-days
Yes
Yes
No
No
100000
Cold wave
Fog
Freezing rain
Hail
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing


In [87]:
my_dict_2

{'Nicaragua': [0],
 'Brazil': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All of Brazilian territory is covered by Early Warnings'],
 'Kuwait': [0],
 'Ireland': ['Cold wave',
  'Fog',
  'Freezing rain',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2-days',
  'Yes',
  'Yes',
  'No',
  'No',
  '100000'],
 'Poland': ['Cold wave',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Yes',
  'No',
  'Yes',
  'up to 4 day

array 3

In [98]:
my_dict_3 = {}

In [99]:
for item in range(2,4):
    print(item)

2
3


this code run

In [ ]:
for country in slice_list[2]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_3[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_3[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



In [102]:
my_dict_3

{'Morocco': ['Cold wave',
  'Dust storm/Sandstorm',
  'Hail',
  'Heat wave',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  '3 - jours',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 'Kiribati': [0],
 'Seychelles': ['Drought/Dry spell',
  'Flood',
  'Lightning',
  'Storm surge/Coastal flood',
  'Tropical cyclone',
  'Tsunami',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No'],
 'Uzbekistan': [0],
 'Egypt': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'days or 6',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 'Curaçao and Sint Maarten': [0],
 "Democratic People's Republic of Korea": [0],
 'Chad': ['Drought/Dry spell',
  'Dust storm/Sandstorm'

array 4

In [136]:
my_dict_4 = {}

In [138]:
for country in slice_list[3]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_4[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_4[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Chile
country been selected successfully!
Cold wave
Drought/Dry spell
Fog
Frost
Hail
Haze/Smoke
Heat wave
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Volcanic ash
Yes
No
Yes
5 días
Yes
Yes
Yes
Yes
Yes
No
Cold wave
Drought/Dry spell
Fog
Frost
Hail
Haze/Smoke
Heat wave
High UV radiation
Icing
Rain/Wet Spell
Snow
Snowstorm
Thunderstorms/Squall lines
Tornado
Volcanic ash
Yes
No
Yes
5 días
Yes
Yes
Yes
Yes
Yes
No
-----
Zambia
country been selected successfully!
No warnings found for this country.
-----
British Caribbean Territories - Anguilla
country been selected successfully!
No warnings found for this country.
-----
Latvia
country been selected successfully!
No warnings found for this country.
-----
Panama
country been selected successfully!
Drought/Dry spell
Flood
High UV radiation
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Tropical cyclone
Wild land fire/Forest fire
Wind
Yes
No
Yes
Aviso de Vigilancia son 72 horas de anticipación y de

In [139]:
my_dict_4

{'Chile': ['Cold wave',
  'Drought/Dry spell',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High UV radiation',
  'Icing',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Volcanic ash',
  'Yes',
  'No',
  'Yes',
  '5 días',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No'],
 'Zambia': [0],
 'British Caribbean Territories - Anguilla': [0],
 'Latvia': [0],
 'Panama': ['Drought/Dry spell',
  'Flood',
  'High UV radiation',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Tropical cyclone',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'No',
  'Yes',
  'Aviso de Vigilancia son 72 horas de anticipación y de Advertencia de dos a tres horas de anticipación y se actualiza cada tres horas.',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'No',
  'El 75 de la población.'],
 'Ukraine': ['Avalanche',
  'Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'F

array 5

In [140]:
my_dict_5 = {}

In [141]:
for country in slice_list[4]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_5[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_5[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Cameroon
country been selected successfully!
Cold wave
Dust storm/Sandstorm
Fog
Haze/Smoke
Heat wave
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
No
Yes
Yes
72 heures
Yes
Yes
No
Cold wave
Dust storm/Sandstorm
Fog
Haze/Smoke
Heat wave
Landslide/Mudslide & Debris flow
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Wind
No
Yes
Yes
72 heures
Yes
Yes
No
-----
Croatia
country been selected successfully!
Cold wave
Drought/Dry spell
Flood
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Yes
Yes
Yes
2 days
Yes
Yes
Yes
Yes
Yes
Yes
100
Cold wave
Drought/Dry spell
Flood
Fog
Freezing rain
Frost
Hail
Haze/Smoke
Heat wave
High Seas/Rogue waves etc.
High UV radiation
Icing
Lightning
Pollen pollution/Polluted air
Rain/Wet Spell
Yes
Yes
Yes
2 days
Yes
Yes
Yes
Yes
Yes
Yes
100
-----
Hong Kong, China
country been selecte

In [142]:
my_dict_5

{'Cameroon': ['Cold wave',
  'Dust storm/Sandstorm',
  'Fog',
  'Haze/Smoke',
  'Heat wave',
  'Landslide/Mudslide & Debris flow',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  '72 heures',
  'Yes',
  'Yes',
  'No'],
 'Croatia': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Pollen pollution/Polluted air',
  'Rain/Wet Spell',
  'Yes',
  'Yes',
  'Yes',
  '2 days',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100'],
 'Hong Kong, China': ['Cold wave',
  'Dust storm/Sandstorm',
  'Flood',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Landslide/Mudslide & Debris flow',
  'Lightning',
  'Rain/Wet Spell',
  'Storm surge/Coastal flood',
  'Thunderstorms/Squall lines',
  'Tornado',
  'Yes

In [151]:
slice_list[5]

['Georgia',
 'Thailand',
 'Cambodia',
 'Guyana',
 'Afghanistan',
 'Bangladesh',
 'Iraq',
 'Slovakia',
 'Costa Rica']

save the progress, there has been 5 dictionary 
- combine them
- add dates 
- save as a csv file 

In [153]:
import pandas as pd

In [157]:
my_dict_2

{'Nicaragua': [0],
 'Brazil': ['Cold wave',
  'Drought/Dry spell',
  'Flood',
  'Fog',
  'Frost',
  'Hail',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Wind',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All of Brazilian territory is covered by Early Warnings'],
 'Kuwait': [0],
 'Ireland': ['Cold wave',
  'Fog',
  'Freezing rain',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'High UV radiation',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '2-days',
  'Yes',
  'Yes',
  'No',
  'No',
  '100000'],
 'Poland': ['Cold wave',
  'Drought/Dry spell',
  'Extra-tropical cyclone',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Yes',
  'No',
  'Yes',
  'up to 4 day

In [169]:
combined_dict = {**my_dict, **my_dict_2, **my_dict_3, **my_dict_4, **my_dict_5}

combined_dict

{'Papua New Guinea': [0],
 'Singapore': ['Drought/Dry spell',
  'Haze/Smoke',
  'Heat wave',
  'High Seas/Rogue waves etc.',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/Squall lines',
  'Tropical cyclone',
  'Volcanic ash',
  'No',
  'No',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'All'],
 'Sao Tome and Principe': [0],
 'Czechia': ['Cold wave',
  'Flood',
  'Fog',
  'Freezing rain',
  'Frost',
  'Hail',
  'Heat wave',
  'Icing',
  'Lightning',
  'Rain/Wet Spell',
  'Snow',
  'Snowstorm',
  'Thunderstorms/Squall lines',
  'Wild land fire/Forest fire',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  '100000'],
 'Djibouti': ['Dust storm/Sandstorm',
  'Rain/Wet Spell',
  'Wind',
  'Yes',
  'Yes',
  'Yes',
  '6HEURES',
  'Yes',
  'Yes',
  'Yes'],
 'Sri Lanka': [0],
 'Burkina Faso': ['Cold wave',
  'Drought/Dry spell',
  'Dust storm/Sandstorm',
  'Haze/Smoke',
  'Heat wave',
  'Lightning',
  'Rain/Wet Spell',
  'Thunderstorms/S

In [176]:
countries, warnings = zip(*combined_dict.items())
df = pd.DataFrame({'Country': countries, 'Warnings': warnings})
df

,Country,Warnings
0,Papua New Guinea,[0]
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig..."
2,Sao Tome and Principe,[0]
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ..."
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y..."
5,Sri Lanka,[0]
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand..."
7,Brunei Darussalam,[0]
8,British Caribbean Territories - Montserrat,[0]
9,Nicaragua,[0]


adding dates

In [177]:
df["date"]= "09/26/2023"

df

,Country,Warnings,date
0,Papua New Guinea,[0],09/26/2023
1,Singapore,"[Drought/Dry spell, Haze/Smoke, Heat wave, Hig...",09/26/2023
2,Sao Tome and Principe,[0],09/26/2023
3,Czechia,"[Cold wave, Flood, Fog, Freezing rain, Frost, ...",09/26/2023
4,Djibouti,"[Dust storm/Sandstorm, Rain/Wet Spell, Wind, Y...",09/26/2023
5,Sri Lanka,[0],09/26/2023
6,Burkina Faso,"[Cold wave, Drought/Dry spell, Dust storm/Sand...",09/26/2023
7,Brunei Darussalam,[0],09/26/2023
8,British Caribbean Territories - Montserrat,[0],09/26/2023
9,Nicaragua,[0],09/26/2023


save as a pickle

In [178]:
# # # Save to a file
# # Save DataFrame to a pickle file
# df.to_pickle('dataframe.pkl')

In [179]:
df_loaded = pd.read_pickle('dataframe.pkl')

where we left!!! 

In [149]:
my_dict_6 = {}

In [150]:
for country in slice_list[5]:
    print(country)
    time.sleep(20)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(20)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict_6[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict_6[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(15)



Georgia
country been selected successfully!
No warnings found for this country.
-----
Thailand
country been selected successfully!
Drought/Dry spell
Flood
Hail
Haze/Smoke
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
N/A
Drought/Dry spell
Flood
Hail
Haze/Smoke
High Seas/Rogue waves etc.
High UV radiation
Icing
Landslide/Mudslide & Debris flow
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Tsunami
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
Yes
Yes
N/A
-----
Cambodia
country been selected successfully!
Drought/Dry spell
Flood
Lightning
Rain/Wet Spell
Storm surge/Coastal flood
Thunderstorms/Squall lines
Tropical cyclone
Wind
Yes
Yes
Yes
No
Yes
Yes
Yes
No
It varies depending on the territory. There is no uniform answer to this question but the urban areas have more people at 

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=117.0.5938.88)
Stacktrace:
0   chromedriver                        0x000000010510bed8 chromedriver + 5091032
1   chromedriver                        0x0000000105102c43 chromedriver + 5053507
2   chromedriver                        0x0000000104c9e366 chromedriver + 447334
3   chromedriver                        0x0000000104ce2fd1 chromedriver + 729041
4   chromedriver                        0x0000000104d14662 chromedriver + 931426
5   chromedriver                        0x0000000104ce0ba8 chromedriver + 719784
6   chromedriver                        0x0000000104d1481e chromedriver + 931870
7   chromedriver                        0x0000000104d306e1 chromedriver + 1046241
8   chromedriver                        0x0000000104d14433 chromedriver + 930867
9   chromedriver                        0x0000000104cdf042 chromedriver + 712770
10  chromedriver                        0x0000000104ce026e chromedriv

In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(5)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)



10 country

In [ ]:
test_list = [ 'Solomon Islands', 'Ireland', 'Romania', 'Guyana', 'Turkmenistan', 'British Caribbean Territories', 'Monaco', 'United States of America', 'Cyprus']
print(len(test_list))
my_dict = {}

In [ ]:
for country in test_list:
    print(country)
    time.sleep(12)
    success = select_country_from_dropdown(driver_instance, wait, country)
    
    if success:
        print("country been selected successfully!")
        time.sleep(5)
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
        print("-----")
    else:
        print("Failed to select the country!")
    time.sleep(10)

## 9 adjusting so we can run the full loop

In [ ]:
test_list = ['Solomon Islands', 'Ireland', 'Romania', 'Guyana']
print(len(test_list))
my_dict = {}

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, "Benin")
if success:
    print("has been selected successfully!")
else:
    print("Failed to select!")

In [ ]:
success = select_country_from_dropdown(driver_instance, wait, country)

In [ ]:
for country in test_list:
    print(country)
    success = select_country_from_dropdown(driver_instance, wait, country)
    if success:
        print("country been selected successfully!")
        warnings = saveWarnings(driver_instance, '.title')
        
        # Check if there are any warnings
        if warnings:
            for warning in warnings:
                print(warning.text)
        
            # Fetch the warnings
            warnings_elements = saveWarnings(driver_instance, '.title')
            
            # Extract the text from each warning element and store it in a list
            warnings_texts = [warning.text for warning in warnings_elements]
            
            # Print each warning's text
            for warning_text in warnings_texts:
                print(warning_text)

            # Save the list of warning texts to the dictionary with country as the key
            my_dict[country] = warnings_texts

        else:
            print("No warnings found for this country.")
            my_dict[country] = [0] 
    else:
        print("Failed to select the country!")
        
